In [2]:
import os
import gc

import pandas as pd
import numpy as np
import torch

from transformers import PatchTSTForPrediction

from torch.utils.data import DataLoader
from datasets import Dataset

2025-11-04 23:15:09.537422: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
data = "coin"

output_dir = "saved_models"
log_dir = os.path.join('logstf', data)

loss_name = "mse"

num_train_epochs = 300
model_num = 1
model_path = "./saved_models"
learning_rate = 1e-6

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
## target domain
target_X = pd.read_csv(f"../data/{data}/train_input_7.csv").iloc[:, 1:].values.astype(np.float32)
target_y = pd.read_csv(f"../data/{data}/train_output_7.csv").iloc[:, 1:].values.astype(np.float32)

target_X_val = target_X[-round(target_X.shape[0] * 0.2):, :].astype(np.float32)
target_y_val = target_y[-round(target_y.shape[0] * 0.2):].astype(np.float32)
target_X = target_X[:-round(target_X.shape[0] * 0.2), :].astype(np.float32)
target_y = target_y[:-round(target_y.shape[0] * 0.2)].astype(np.float32)

test_X  = pd.read_csv(f"../data/{data}/val_input_7.csv").iloc[:, 1:].values.astype(np.float32)
test_y  = pd.read_csv(f"../data/{data}/val_output_7.csv").iloc[:, 1:].values.astype(np.float32)

In [5]:
def array_to_dataset(X, y):
    X, y = torch.tensor(X), torch.tensor(y)
    X = X.reshape(-1, X.shape[1], 1)
    y = y.reshape(-1, y.shape[1], 1)

    dataset = torch.utils.data.TensorDataset(X, y)

    return dataset

train_dataset = array_to_dataset(target_X, target_y)
val_dataset = array_to_dataset(target_X_val, target_y_val)
test_dataset = array_to_dataset(test_X, test_y)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size = 8, shuffle = True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size = 64)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size = 64)

In [ ]:
for k in range(1, model_num+1):
    current_path = os.path.join(model_path, f"model_{loss_name}_{k}.pth")

    backbone_model = PatchTSTForPrediction.from_pretrained(os.path.join(model_path, "PatchTSTBackbone")).to(device)
    backbone_model.load_state_dict(torch.load(current_path))
    backbone = backbone_model.model     ## 헤드 제거

In [19]:
class TransferAll(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.net = torch.nn.Sequential(
            torch.nn.Linear(24, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 24)
        )

    def forward(self, x):
        yhat = self.net(x.prediction_outputs.squeeze())

        return yhat
    
model_instance = torch.nn.Sequential(
    backbone_model,
    TransferAll()
).to(device)

In [ ]:
T_OUT = target_y.shape[1]
C_NEW = 128

In [ ]:
# class TransferMLP(torch.nn.Module):
#     def __init__(self, body, t_out, c_new):
#         super().__init__()
#         self.body = body
#         self.t_out = t_out
#         self.c_new = c_new
#         body_out_features = body.encoder.layers[-1].ff[-1].out_features

#         self.flatten = torch.nn.Flatten(start_dim = 2, end_dim = -1)
#         self.adapter = torch.nn.Linear(body_out_features*7, self.t_out * self.c_new)  ## Dense(128)

#         self.head = torch.nn.Sequential(
#             torch.nn.Dropout(0.2),
#             torch.nn.Linear(128, 64),
#             torch.nn.Dropout(0.2),
#             torch.nn.Linear(64, 1)
#         )

#     def forward(self, x):
#         features = self.body(past_values=x).last_hidden_state
#         flat_feat = self.flatten(features)  ## (B, body_out_features*10)
#         adapted_feat = self.adapter(flat_feat)
#         head_input = adapted_feat.view(-1, self.t_out, self.c_new)  ## (B, 24, 128)
#         output = self.head(head_input)

#         return output


# model_instance = TransferMLP(backbone, T_OUT, C_NEW).to(device)

In [ ]:
## custom loss function
def SMAPE(yhat, y):
    numerator = 100*torch.abs(y - yhat)
    denominator = (torch.abs(y) + torch.abs(yhat))/2
    smape = torch.mean(numerator / denominator)
    return smape

def MAPE(y_pred, y_true, epsilon=1e-7):
    denominator = torch.clamp(torch.abs(y_true), min=epsilon)       ## 분모에 0이 들어오는 것을 방지
    abs_percent_error = torch.abs((y_true - y_pred) / denominator)

    return torch.mean(100. * abs_percent_error)


class MASE(torch.nn.Module):
    def __init__(self, training_data, period = 1):
        super().__init__()
        ## 원본 코드 구현, 사실상 MAE와 동일, 잘못 짜여진 코드, 일단은 하던대로 할 것.
        self.scale = torch.mean(torch.abs(torch.tensor(training_data[period:] - training_data[:-period])))
    
    def forward(self, yhat, y):
        error = torch.abs(y - yhat)
        return torch.mean(error) / self.scale

In [ ]:
optimizer = torch.optim.Adam(model_instance.parameters(), lr = learning_rate)
log_data = []

if loss_name == "mse":
    loss_fn = torch.nn.MSELoss()
elif loss_name == "mae":
    loss_fn = torch.nn.L1Loss()
elif loss_name == "SMAPE":
    loss_fn = SMAPE
elif loss_name == "mape":
    loss_fn = MAPE
elif loss_name == "MASE":
    loss_fn = MASE(target_y, target_y.shape[1])
else:
    raise Exception("Your loss name is not valid.")

## early stopping
PATIENCE = 10
best_val_loss = np.inf
patience_counter = 0

for epoc in range(num_train_epochs):
    model_instance.train()

    total_train_loss = 0

    for X, y in train_dataloader:
        X, y = X.to(device), y.to(device)

        optimizer.zero_grad()
        yhat = model_instance(X)
        loss = loss_fn(yhat, y)
        loss.backward()
        optimizer.step()

        total_train_loss += loss.item()*X.shape[0]

    avg_train_loss = total_train_loss/len(train_dataloader.dataset)

    model_instance.eval()

    with torch.no_grad():
        yys = []
        yyhats = []

        for XX, yy in val_dataloader:
            XX = XX.to(device)
            yys.append(yy.to(device))
            yyhats.append(model_instance(XX))

        yyhat = torch.concat(yyhats)
        yy = torch.concat(yys)

        val_loss = loss_fn(yyhat, yy).item()

    print(f"Epoch {epoc+1}/{num_train_epochs} | Train Loss: {avg_train_loss:.6f}\t\t Val Loss: {val_loss:.6f}")

    log_data.append({"epoch": epoc, "loss": avg_train_loss, "eval_loss": val_loss})

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_state_dict = model_instance.state_dict()   ## 저장 없이 결과물만 산출...
        patience_counter = 0
    else:
        patience_counter += 1

    if patience_counter >= PATIENCE:
        break

Epoch 1/300 | Train Loss: 26763.466331		 Val Loss: 4877.003418
Epoch 2/300 | Train Loss: 26759.033359		 Val Loss: 4875.743164
Epoch 3/300 | Train Loss: 26754.698482		 Val Loss: 4873.718750
Epoch 4/300 | Train Loss: 26749.697272		 Val Loss: 4872.794434
Epoch 5/300 | Train Loss: 26745.558094		 Val Loss: 4870.468262
Epoch 6/300 | Train Loss: 26740.279631		 Val Loss: 4869.871582
Epoch 7/300 | Train Loss: 26735.570907		 Val Loss: 4868.115723
Epoch 8/300 | Train Loss: 26730.364849		 Val Loss: 4866.382812
Epoch 9/300 | Train Loss: 26725.104793		 Val Loss: 4864.151855
Epoch 10/300 | Train Loss: 26719.639544		 Val Loss: 4862.168457
Epoch 11/300 | Train Loss: 26714.222724		 Val Loss: 4861.132324
Epoch 12/300 | Train Loss: 26709.405986		 Val Loss: 4859.150391
Epoch 13/300 | Train Loss: 26702.729401		 Val Loss: 4855.653809
Epoch 14/300 | Train Loss: 26697.281669		 Val Loss: 4853.259766
Epoch 15/300 | Train Loss: 26690.540184		 Val Loss: 4850.000977
Epoch 16/300 | Train Loss: 26683.998588		 Val Los

In [ ]:
pd.DataFrame(log_data).to_csv(os.path.join(log_dir, f"transfer_{loss_name}_lr{learning_rate}_run{1}.csv"))

model_instance.load_state_dict(best_state_dict)

<All keys matched successfully>

In [ ]:
with torch.no_grad():
    yys = []
    yyhats = []

    for XX, yy in test_dataloader:
        XX = XX.to(device)
        yys.append(yy.to(device))
        yyhats.append(model_instance(XX))

    yyhat = torch.concat(yyhats)
    yy = torch.concat(yys)

    test_loss = loss_fn(yyhat, yy)

In [ ]:
mseLoss = torch.nn.MSELoss()
maeLoss = torch.nn.L1Loss()

def smape(yy, yyhat):
    numerator = 100*abs(yy - yyhat)
    denominator = (abs(yy) + abs(yyhat))/2
    smape = torch.mean(numerator / denominator)
    return smape

print(f"test RMSE: {torch.sqrt(mseLoss(yyhat, yy))}")
print(f"test MAE: {maeLoss(yyhat, yy)}")
print(f"test SMAPE: {smape(yy, yyhat)}")

test RMSE: 58.111000061035156
test MAE: 54.474143981933594
test SMAPE: 53.27073287963867
